- latest `trl` version
    - https://huggingface.co/docs/trl/main/en/grpo_trainer

In [1]:
from IPython.display import Image

In [2]:
from datasets import load_dataset
dataset = load_dataset("trl-lib/tldr", split="train")

In [3]:
dataset

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 116722
})

## GRPOConfig & GRPOTrainer

In [4]:
import os
os.environ["NCCL_P2P_DISABLE"] = "1"
os.environ["NCCL_IB_DISABLE"] = "1"

In [5]:
from trl import GRPOConfig, GRPOTrainer

[2025-02-15 08:36:32,909] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.4
 [WARNING]  using untested triton version (3.0.0), only 1.0.0 is known to be compatible


/home/whaow/anaconda3/lib/python3.10/site-packages/deepspeed/runtime/zero/linear.py:49: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, input, weight, bias=None):
/home/whaow/anaconda3/lib/python3.10/site-packages/deepspeed/runtime/zero/linear.py:67: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, grad_output):
/home/whaow/anaconda3/lib/python3.10/site-packages/vllm/connections.py:8: RuntimeWarning: Failed to read commit hash:
No module named 'vllm._version'
  from vllm.version import __version__ as VLLM_VERSION


In [10]:
from rich.pretty import pprint
# pprint(GRPOConfig(output_dir='test'))

- GRPOConfig
    - num_generations=8,
    - old
        - per_device_train_batch_size=1, * gradient_accumulation_steps=8,
            - per_device_train_batch_size * gradient_accumulation_steps * world_size ==> train_batch
    - new: https://github.com/huggingface/trl/pull/2776#issue-2833772774
        - per_device_train_batch_size:
            - it now represents the number of generations per device.
        - per_device_train_batch_size/num_generations * gradient_accumulation_steps
            - per_device_train_batch_size/num_generations: prompts per device
            - 也因此要求，per_device_train_batch_size 必须能被 num_generations 整除；

## running


> dataset: 7473;

- 单 4090:
    ```
    export CUDA_VISIBLE_DEVICES=0
    python grpo_demo.py
    ```
    - 4/4 * 4 => 4,
        - 1868;
    - 13 小时；
- 双 4090，ddp （accelerate map）
    - (4/4 * 4 * 2) => 8;
        - 934;
    - < 10小时；
- 双 4090，deepspeed stage 3;

## GRPO

In [7]:
# bs = 2
# G = 4
Image(url='https://huggingface.co/datasets/trl-lib/documentation-images/resolve/main/grpo_visual.png', width=400)

- GRPO is an **online learning**(on-policy) algorithm, meaning it improves iteratively by using the data generated by the trained model itself during training.
- four main steps:
    - Generating completions,
        - At each training step, we sample a batch of prompts and generate a set of $G$ completions(`num_generations`) for each prompt (denoted as $o_i$).
    - computing the advantage,
        - $\hat A_{i,t}=\frac{r_i-\mu(\mathbf r)}{\sigma(\mathbf r)}$
        - Outcome supervision provides the normalized reward at the end of each output $o_i$ and sets the advantages $\hat A_{i,t}$ of all tokens in the output as the normalized reward
    - **estimating** the KL divergence, (token-level see the figure)
        - https://huggingface.co/docs/trl/main/en/grpo_trainer#estimating-the-kl-divergence
        - `per_token_kl = torch.exp(ref_per_token_logps - per_token_logps) - (ref_per_token_logps - per_token_logps) - 1`
    - and computing the loss.
        - $\pi_{ref}, (\pi_{old}, \pi_\theta)$
        - https://github.com/huggingface/trl/issues/2608
```
# x - x.detach() allows for preserving gradients from x
per_token_loss = torch.exp(per_token_logps - per_token_logps.detach()) * advantages.unsqueeze(1)
per_token_loss = -(per_token_loss - self.beta * per_token_kl)
loss = ((per_token_loss * completion_mask).sum(dim=1) / completion_mask.sum(dim=1)).mean()
```

### reward function

```
def reward_func(completions, **kwargs):
    """Reward function that gives higher scores to longer completions."""
    return [float(len(completion)) for completion in completions]
```
- completions
    - `[bs, G]`
- 20
    - https://github.com/huggingface/trl/issues/2771

### pipeline

> qwen2.5 vocab size: 151936

- prompts => tokenizer.apply_chat_template => model.generate
    - prompt_completion_ids = prompt_ids + prompt_completion_ids
    - rewards_per_func: (n_prompts, len(reward_funcs))
    - ref_per_token_logps($\pi_{ref}(q, o)$), per_token_logps ($\pi_\theta(q,o)$)
        - completion token level
            - selective_log_softmax(logits, index)
                - logits.shape: (n_prompts, n_completion, n_vocab)
                - index.shape: (n_prompts, n_complection)
                - => (n_prompts, n_complection)
$$
\exp(\log{\pi'}-\log{\pi})=\frac{\pi'}{\pi}
$$

- 目前的实现只有 $\pi_{\theta}, \pi_{ref}$，没有 $\pi_{old}$
    - https://github.com/huggingface/trl/issues/2608
        - The policy model only has **a single update** following each exploration stage. (deepseekmath)
        - $\pi_\theta$ 每次（rollout a group generations）只进行一次更新，而不是多次更新；
            - 对应 `for step = 1, . . . , M do` (M == 1)
    - $\pi_{old}=\pi_\theta$
    - `torch.exp(per_token_logps - per_token_logps.detach()) * advantages.unsqueeze(1)`
        - per_token_logps.detach() 不参与计算图的梯度计算；
    - 没有用到 clip，只有 $\frac{\pi}{\pi_{old}}A=1\cdot A$（ratio * advantage）
        - ratio = 1，一定在 $(1-\epsilon, 1+\epsilon)$ 之间的；

### 显存分析

- models, data: 从这两个角度分析算法流程以及可能的显存占用；

### PPO vs. GRPO

在 RLHF 中，我们需要把人类反馈或reward model 对整个序列的打分（例如，一次对话的最终质量分）融合到强化学习训练中。此外，为了让模型在训练时不要偏离参考策略（reference model）太远，我们常常还会引入一个基于 KL 的惩罚项。

因此，在 PPO 里面，最关键的变化在于——如何构造每个 token（每个时间步）的奖励 $r_t$
 。这往往通过下面两步完成：

## metrics


### training monitor

- You should rely mostly on the reward. And keep an eye on the generations (risk of reward hacking)
    - https://github.com/huggingface/trl/issues/2703